In [20]:
%cd ..

/home/chanwutk/code


In [21]:
def is_notebook() -> bool:
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True   # Jupyter notebook or qtconsole
        elif shell == 'TerminalInteractiveShell':
            return False  # Terminal running IPython
        else:
            return False  # Other type (?)
    except NameError:
        return False      # Probably standard Python interpreter

In [22]:
import json
import os
import pickle
import traceback

import numpy as np

if is_notebook():
    from tqdm.notebook import tqdm
else:
    from tqdm import tqdm

In [3]:
from optimized_ingestion.camera_config import camera_config
from optimized_ingestion.payload import Payload
from optimized_ingestion.pipeline import Pipeline
from optimized_ingestion.video import Video

In [4]:
# Stages
from optimized_ingestion.stages.in_view import InView, InViewOld

In [7]:
NUSCENES_PROCESSED_DATA = "NUSCENES_PROCESSED_DATA"

In [8]:
print(NUSCENES_PROCESSED_DATA in os.environ)
print(os.environ['NUSCENES_PROCESSED_DATA'])

True
/home/chanwutk/data/processed/full-dataset/trainval


In [9]:
DATA_DIR = os.environ[NUSCENES_PROCESSED_DATA]
with open(os.path.join(DATA_DIR, "videos", "frames.pkl"), "rb") as f:
    videos = pickle.load(f)

In [10]:
with open(os.path.join(DATA_DIR, 'cities.pkl'), 'rb') as f:
    cities = pickle.load(f)

In [17]:
def run_benchmark(pipeline, filename, ignore_error=False):
    keep = {}
    failed_videos = []
    names = cities['boston-seaport'][:]
    filtered_videos = [(n, v) for n, v in videos.items() if n[6:10] in names]
    for i, (name, video) in tqdm(enumerate(filtered_videos), total=len(filtered_videos)):
        if name[6:10] not in names:
            continue

        try:
            video_filename = video['filename']
            if not video_filename.startswith('boston'): continue
            # if not video_filename.startswith('boston') or 'FRONT' not in name: continue

            frames = Video(
                os.path.join(DATA_DIR, "videos", video["filename"]),
                [camera_config(*f, 0) for f in video["frames"]],
            )

            output = pipeline.run(Payload(frames))
            keep[name] = str(output.keep)
        except Exception as e:
            if ignore_error:
                message = str(traceback.format_exc())
                failed_videos.append((name, message))
                print(e)
                print(message)
                print("------------------------------------------------------------------------------------")
                print()
                print()
            else:
                raise e
    with open(f"./outputs/{filename}-keep.json", "w") as f:
        json.dump(keep, f, indent=2)

    with open(f"./outputs/{filename}-failed-videos.json", "w") as f:
        json.dump(failed_videos, f, indent=2)

    with open(f"./outputs/{filename}-performance.json", "w") as f:
        performance = [
            {
                "stage": stage.classname(),
                "runtimes": stage.runtimes,
            }
            for stage
            in pipeline.stages
        ]
        json.dump(performance, f, indent=2)

In [18]:
type_filter = ['bicycle', 'motorcycle', 'car', 'truck', 'bus']

In [19]:
p0 = Pipeline()
p0.add_filter(InViewOld(50, 'intersection'))

run_benchmark(p0, 'test-in-view-old', ignore_error=True)

  0%|          | 0/2802 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
p1 = Pipeline()
p1.add_filter(InView(50, 'intersection'))

run_benchmark(p1, 'test-in-view-new', ignore_error=True)

In [ ]:
p2 = Pipeline()
p2.add_filter(InViewOld(50, 'intersection', min_distance=True))

run_benchmark(p2, 'test-in-view-old-min-distance', ignore_error=True)